In [2]:
import numpy as np
import pandas as pd
df = pd.read_excel('house_purchase.xlsx')

# Check the columns of the DataFrame
print(df.columns)

# Define the entropy function
def entropy(s):
    p = [(s == c).mean() for c in set(s)]
    return sum([-pi * np.log2(pi) for pi in p if pi != 0])


# Define the information gain function
def info_gain(data, attribute, target):
    totalE = entropy(data[target])
    vals, counts = np.unique(data[attribute], return_counts=True)
    wt = np.sum([(counts[i] / np.sum(counts)) * entropy(data.where(data[attribute] == val).dropna()[target]) for i, val in enumerate(vals)])
    return totalE - wt

# Define the ID3 algorithm
def id3(data, og, features, target, parent_node_class=None):
    if len(np.unique(data[target])) <= 1:
        return np.unique(data[target])[0]
    elif len(data) == 0:
        return np.unique(og[target])[np.argmax(np.unique(og[target], return_counts=True)[1])]
    elif len(features) == 0:
        return parent_node_class
    else:
        parent_node_class = np.unique(data[target])[np.argmax(np.unique(data[target], return_counts=True)[1])]
    item = [info_gain(data, feature, target) for feature in features]
    bestI = np.argmax(item)
    bestF = features[bestI]
    tree = {bestF: {}}
    features = [i for i in features if i != bestF]
    for value in np.unique(data[bestF]):
        value = value
        sub = data.where(data[bestF] == value).dropna()
        subtree = id3(sub, df, features, target, parent_node_class)
        tree[bestF][value] = subtree
    return tree

# Define the predict function
def predict(query, tree):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]]
            except:
                return "Cannot make a prediction"
            result = tree[key][query[key]]
            if isinstance(result, dict):
                return predict(query, result)
            else:
                return result

# Define the query dictionary
query = {'Age': 30, 'Income': 50000, 'Gender': 'Male', 'MaritalStatus': 'Single'}

# Define features and target
features = ['Age', 'Income', 'Gender', 'MaritalStatus']
target = 'Buy'

# Build the decision tree
tree = id3(df, df, features, target)

# Predict the class label
prediction = predict(query, tree)
print("Predicted class label:", prediction)


Index(['Age', 'Income', 'Gender', 'MaritalStatus', 'Buy'], dtype='object')
Predicted class label: N
